In [3]:
import requests
from datetime import datetime, timedelta
from typing import List
import json

from optimization import improve_graph

def get_task_goals(
    base_url: str = "https://stackvm.tidb.ai/",
   last_hours: int = 2
) -> List[str]:
    try:
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(hours=last_hours)
        
        url = f"{base_url}/api/tasks/evaluation"
        params = {
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat()
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        tasks = response.json()
        
        goals = [task.get("goal", "") for task in tasks if task.get("goal")]
        
        return goals
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching task goals: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return []
    except Exception as e:
        print(f"Unknown error: {e}")
        return []

goals = get_task_goals()
for goal in goals:
    improve_graph(goal)

Found new issues 0, total issues 121
Identified 119 valid issues
issue is resolved {('redundancy_entity', (33346, 363328, 1380022, 1380024, 1410020, 1500022)): True, ('redundancy_entity', (35666, 660260, 1500021)): True, ('redundancy_entity', (50205, 361880, 1380027, 1440021, 1500020)): True, ('redundancy_entity', (30178, 1500023)): True, ('redundancy_entity', (67446, 361880, 1410021)): True, ('redundancy_entity', (41603, 59518, 900158)): True, ('redundancy_entity', (34673, 50555, 53518, 53534)): True, ('redundancy_entity', (58191, 900373)): True, ('entity_quality_issue', (241263,)): True, ('entity_quality_issue', (43094,)): True, ('entity_quality_issue', (41041,)): True, ('entity_quality_issue', (50555,)): True, ('entity_quality_issue', (57765,)): True, ('redundancy_entity', (32646, 34144, 39735, 51787, 67835, 243832, 720293)): True, ('redundancy_entity', (36675, 240202)): True, ('redundancy_entity', (34877, 59951)): True, ('redundancy_relationship', (34335, 40987, 43623, 43642, 24074